In [1]:
# Logistic regression learning algorithm using TensorFlow library.
# Base example from Aymeric Damien, https://github.com/aymericdamien/TensorFlow-Examples/

# Currently a loose idea of what we need to make a model for 
# Still need to read in data
# MNIST data kept in comments just in case we need to retest something

In [2]:
import tensorflow as tf
import numpy as np
import json
import os

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
# read in data from chosen files
data = []
labels = []
# correct user is specified for which labels will be marked as '1'
correctUser = "hannah"
path = './data'

# read in data from files
for filename in os.listdir(path):
    fp = open("./data/"+filename, "r")
    j = json.load(fp)

    # after file is open put the json data into lists for training
    for user in j:
        if user == correctUser:
            for attempt in j[user]:
                templist = []
                for key in attempt["Keys"]:
                    templist.append(key["Fight"])
                    templist.append(key["Dwell"])
                templist.append(attempt["Metadata"][0]["Total Time"])
                print "-"
                data.append(templist)
                labels.append(1)
        else:
            for attempt in j[user]:
                templist = []
                for key in attempt["Keys"]:
                    templist.append(key["Fight"])
                    templist.append(key["Dwell"])
                templist.append(attempt["Metadata"][0]["Total Time"])
                print "-"
                data.append(templist)
                labels.append(0)

data = np.asarray(data)
labels = tf.one_hot(labels,2)


-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-


In [11]:
#normalize data input, feature scaling for better learning
#theres probably a better way to do this but it works
data = data.astype(float)
for i in range(len(data)):
    data[i] = (data[i] - np.mean(data[i])) / np.std(data[i])
    
#normThis = []
#separate same values into separate lists to make math better
#for j in range(len(data[0])):
#    normThis.append([i[0] for i in data])
    
#print normThis[:2]
#do the standardization
#for i in range(len(normThis)):
#    normThis[i] = (normThis[i] - np.mean(normThis[i])) / np.std(normThis[i])

#print normThis[:2]

    

In [17]:
# Parameters
learning_rate = 0.2
training_runs = 3000


# tf Graph Input example from MNIST
#x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
#y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# tf Graph Input for PIN
x = tf.placeholder(tf.float32, [None, 9]) # each entry has 
y = tf.placeholder(tf.float32, [None, 2]) # one hot vector should be the output

# Set model weights
W1 = tf.Variable(tf.random_normal([9, 2]))
b1 = tf.Variable(tf.random_normal([2]))
#W1 = tf.Variable(tf.random_normal([9, 20]))
#b1 = tf.Variable(tf.random_normal([20]))
#W2 = tf.Variable(tf.zeros([20, 2]))
#b2 = tf.Variable(tf.zeros([2]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W1) + b1)
#z = tf.nn.softmax(tf.matmul(x, W1) + b1)
#pred = tf.nn.softmax(tf.matmul(z, W2) + b2)

# Minimize error using cross entropy
#cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
cost = -tf.reduce_sum(y*tf.log(tf.clip_by_value(pred,1e-10,1.0)))

# Gradient Descent
#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [18]:
sess=tf.InteractiveSession()
tf.global_variables_initializer().run()

l = sess.run(labels)
#print data
#print l

# Training cycle
print "predictions:", tf.argmax(pred.eval(feed_dict={x:data[:15]}),1).eval()
print pred.eval(feed_dict={x:data[:15]})
for i in range(training_runs):
    # Fit training using json data
    _, error = sess.run([optimizer, cost], {x:data[:15], y:l[:15]})

    # Print error every once and a while to see whats going on
    if (i % 10) == 0:
        #print sess.run(W1)
        #print sess.run(b1)
        print error

print "All done!"
print "predictions:", tf.argmax(pred.eval(feed_dict={x:data[:15]}),1).eval()
print pred.eval(feed_dict={x:data[:15]})
print "actual: ", l[:15]


predictions: [1 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
[[ 0.07430176  0.92569822]
 [ 0.01495064  0.98504931]
 [ 0.01202176  0.98797822]
 [ 0.01026234  0.98973769]
 [ 0.0099756   0.99002433]
 [ 0.5719533   0.42804673]
 [ 0.00889244  0.99110764]
 [ 0.00853297  0.991467  ]
 [ 0.00857953  0.99142051]
 [ 0.00792118  0.99207884]
 [ 0.05573677  0.94426322]
 [ 0.01130966  0.98869032]
 [ 0.00944768  0.99055231]
 [ 0.00887838  0.99112165]
 [ 0.00845151  0.99154848]]
40.1478
16.0712
10.7549
8.95163
8.59628
8.45626
8.32049
8.17968
8.07929
7.99494
7.91223
7.83528
7.76081
7.68871
7.61873
7.55066
7.48445
7.42007
7.3575
7.29673
7.23774
7.18048
7.12493
7.07103
7.01871
6.96793
6.91862
6.8707
6.82412
6.7788
6.73468
6.69169
6.64977
6.60885
6.56888
6.52981
6.49157
6.45412
6.41741
6.3814
6.34604
6.3113
6.27712
6.24349
6.21037
6.17772
6.14552
6.11373
6.08235
6.05133
6.02066
5.99032
5.96029
5.93056
5.9011
5.8719
5.84295
5.81424
5.78575
5.75747
5.7294
5.70152
5.67383
5.64631
5.61897
5.5918
5.56478
5.53792
5.51122
5.4846

In [30]:
# Launch the graph
# not using, this is just a reference
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(10):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/200)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(200)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % 10 == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)
            print sess.run(b1)

    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print "Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]})

x: [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
y: [[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]]


ValueError: Cannot feed value of shape (5, 784) for Tensor u'Placeholder_16:0', which has shape '(?, 9)'